![title](Innomatics-Research-Labs.png)

In [7]:
# Importing necessary libraries.

from bs4 import BeautifulSoup as soup
from requests import get
import pandas as pd
import requests
import re

- Beautiful Soup will help us read this html. It picks the text from the response and parses the information in a     way that makes it easier for us to navigate in its structure and get its contents.
- The requests module allows you to send HTTP requests using Python.

In [116]:
# Assigning the url(Uniform Resource Locator) of website from which data needs to be scarpped.

page=('https://www.makaan.com/hyderabad-residential-property/buy-property-in-hyderabad-city?budget=,&page=3')

In [93]:
# Requesting the data  from url.

response = requests.get('https://www.makaan.com/hyderabad-residential-property/buy-property-in-hyderabad-city?budget=,&page=3')
response

<Response [200]>

In [94]:
# The HTTP request returns a Response Object with all the response data (content, status, etc).

response.text

'<!doctype html> <html lang="en"><head><meta http-equiv="Content-type" content="text/html; charset=utf-8"><title>Property for Sale in Hyderabad | 40861+ Hyderabad Properties for sale</title><meta name="description" content="Search 40861+ Properties for sale in Hyderabad on Makaan.com. Find &#10003;6641+ New Projects for sale. &#10003;13642+ Flats/Apartments. &#10003;3137+ Houses/Villas. Visit Now !"><meta name="keywords" content="Buy Hyderabad Properties, Residential Property for sale in Hyderabad, Property for sale in Hyderabad, Hyderabad Property Sale"><meta name="theme-color" content="#fff" id="themeColor"><meta content="origin" name="referrer"><meta name="p:domain_verify" content="55ce01b3ca93c05fd5a41439a23dd0d9"><meta name="fb:pages" content="155462194517712"><meta name="country" content="India"><meta name="og:type" content="website"><meta name="og:site_name" content="Makaan.com"><meta name="og:image:url" content="http://static.makaan.com/6/12/355/11020788.jpeg"><meta name="viewp

In [117]:
# Creating empty lists in-order to append data scrapped from url's.
# The number of lists depends on the number of features you are extracting from the url.

title=[]
location=[]
price=[]
price_per_sqft=[]
area_in_sqft=[]
building_status=[]

In [118]:
%%time # Returns time take for execution of the cell.

n_pages = 0
for page in range(3,2016):  #The no. of pages you wanted to scrap considering pages available in the url.
    n_pages += 1
    url = 'https://www.makaan.com/hyderabad-residential-property/buy-property-in-hyderabad-city?budget=,'+'&page='+str(page)
    # This for gets us into next page after every iteration.
    r = requests.get(url) # Here r is response data.
    page_html = soup(r.text, 'html.parser') 
    house_containers = page_html.find_all("li",class_="cardholder") # Class containing overall data of a property.
    for data in house_containers:
################################   TITLE   ########################################  

        type_=data.find_all("a",class_="typelink") # Class containing title name of a property.
        for i in type_:
            d=i.text
            title.append(d) # Appending the extracted data into a list.
            
################################ LOCATION  #######################################

        location_=data.find_all("a",class_="loclink")
        for i in location_:
            d=i.text
            r= d.split(',',)[0] # Splitting the obtained text and returning the first element of text.
            location.append(r)
            
################################    PRICE    ######################################

        cost=data.find_all("td",class_="price")
        for i in cost:
            d=i.text
            if 'L' in d:
                a = d.split()[0]
                price.append(float(a)) # Type conversion of obtained data.
            elif 'Cr' in d:
                b = d.split()[0]
                price.append(float(b) * 100) # Coverting CRORES into LAKHS.
            else:  # If no value found in the class return value 0.
                e=0
                price.append(float(e)) 
                
############################   PRICE PER SQFT   ###################################

        rate_sqft=data.find_all('td',class_="lbl rate")
        for i in rate_sqft:
            d=i.text
            res= d.split('/',)[0]
            r=re.sub(",","" ,res) # Substitute no character in the place of ,(comma).
            price_per_sqft.append(r)
            
############################   AREA IN SQFT    ####################################

        area_sqft=data.find_all('td',class_="size")
        for i in area_sqft:
            d=i.text
            area_in_sqft.append(d)
            
##########################   BUILDING STATUS    ###################################

        building_status_=data.find_all("td",class_="val")
        for i in building_status_:
            d=i.text
            building_status.append(d)

###################################################################################

print('You scraped {} pages containing {} properties.'.format(n_pages, len(title)))
# Returns the number of pages scrapped and time taken for scarpping those pages.

You scraped 2013 pages containing 40260 properties.
Wall time: 19min 47s


In [121]:
# Returns the data related to agent rating.

%%time
n_pages = 0
agent_ratings=[]
for page in range(3,2016):
    n_pages += 1
    url = 'https://www.makaan.com/hyderabad-residential-property/buy-property-in-hyderabad-city?budget=,'+'&page='+str(page)
    r = requests.get(url)
    page_html = soup(r.text, 'html.parser')
    house_containers = page_html.find_all("li",class_="cardholder")
    for i,data in enumerate(house_containers):
        agent_rating_=data.find_all('div', class_="rating")
        if i==4:
            pass
        else:
            if len(agent_rating_)==1:
                for i in agent_rating_:
                    d=i.text
                    agent_ratings.append(d)
            else:
                agent_ratings.append(int('0'))  
                
print('You scraped {} pages containing {} properties.'.format(n_pages,len(title)))

You scraped 2013 pages containing 40260 properties.
Wall time: 16min 59s


In [9]:
# Defining a dataframe.

df=pd.DataFrame()

In [291]:
# Returns data in form a dataframe with columns of specified names containing the assigned values from the list.

df['title']=title 
df['location']=location
df['price(L)']=price
df['rate_persqft']=price_per_sqft
df['area_insqft']=area_in_sqft
df['building_status']=building_status
df['agent_rating']=agent_ratings

In [292]:
# Writing the data from dataframe in form of (comma seperated values) CSV file.

d=df.to_csv('projectfinal.csv')

In [293]:
# Reading data from CSV file.

d1=pd.read_csv("projectfinal.csv")

In [296]:
d1.shape

(40260, 8)

### Observation:
- The data we extracted contains **40260** rows and **8** columns.

In [177]:
# Created a copy of csv file 'projectfinal.csv' into excel.(not necessary)
# Reading excel file into a dataframe.

d2=pd.read_excel("dup1.xlsx")

In [178]:
d2.shape

(40260, 8)

### Observation:
- The data we extracted contains **40260** rows and **8** columns same as our original data.

In [179]:
# Returns top 5 rows from dataset (default value of head is 5).

d2.head()

,Unnamed: 0,title,location,price(L),rate_persqft,area_insqft,building_status,agent_rating
0,0,Residential Plot,Maheshwaram,45.00,1111,4050,New,4.6
1,1,Residential Plot,Kondakal,34.00,2361,1440,New,3.7
2,2,Residential Plot,Bibinagar,14.19,944,1503,New,4.9
3,3,Residential Plot,Gachibowli,330.00,3333,9900,New,0.0
4,4,Residential Plot,Shadnagar,6.72,466,1440,New,4.7


In [180]:
# Dropping of the unnecessary column "Unnamed: 0".

d2.drop(columns='Unnamed: 0',axis=1,inplace=True)

In [181]:
d2.head()

,title,location,price(L),rate_persqft,area_insqft,building_status,agent_rating
0,Residential Plot,Maheshwaram,45.00,1111,4050,New,4.6
1,Residential Plot,Kondakal,34.00,2361,1440,New,3.7
2,Residential Plot,Bibinagar,14.19,944,1503,New,4.9
3,Residential Plot,Gachibowli,330.00,3333,9900,New,0.0
4,Residential Plot,Shadnagar,6.72,466,1440,New,4.7


In [182]:
# Returns the column names of dataset.

d2.keys()

Index(['title', 'location', 'price(L)', 'rate_persqft', 'area_insqft',
       'building_status', 'agent_rating'],
      dtype='object')

In [183]:
# Returns the datatypes in the data.

d2.dtypes

title               object
location            object
price(L)           float64
rate_persqft         int64
area_insqft          int64
building_status     object
agent_rating       float64
dtype: object

### Observations:
- Our features in the data set are title, location,price in lakhs, rate_persqft, area_insqft, building_status, agent_rating.
- **title** : Defines the title name of the property, whose datatype is string
- **location** : Defines about the locality(area) of the property, whose datatype is string
- **price(L)** : Defines the price of a property in lakhs, whose datatype is float.
- **rate_persqft** : Defines the rate per unit sqft of property, whose datatype is int.
- **area_insqft** : Defines the total area of property, whose datatype is int.
- **building_status** : Define about the current status of builiding, whose datatype is string
- **agent_rating** : Defines about the rating of agent given by the customers, whose datatype is float.

In [184]:
# Returns aggregrate functions for columns containing numerical data.

d2.describe()

,price(L),rate_persqft,area_insqft,agent_rating
count,40260.000000,40260.000000,40260.000000,40260.000000
mean,65.331670,3297.212072,2360.068728,3.574724
std,126.247236,3850.078324,4190.331350,1.953369
min,0.000000,100.000000,100.000000,0.000000
25%,20.000000,1018.000000,1290.000000,3.400000
50%,37.170000,2222.000000,1674.000000,4.600000
75%,78.630000,4800.000000,2250.000000,4.900000
max,7000.000000,172541.000000,215278.000000,5.000000


### Observations:

 - **price(L)**:
 
    - mean of prices is 65.33 L
    - maximum price is 7000 L i.e, 70 cr.
    - minimum price is 0
    - 25 percentile is 20 L i.e, 25% of prices lie below 20 L.
    - 50 percentile is 37.17 L i.e, 50% of prices lie below 37.17 L.
    - 75 percentile is 78.63 L i.e, 75% of prices lie below 78.63 L.
    - standard deviation(std) is 126.24
 
 - **rate_persqft**:
 
    - mean ratepersqft is 3297.21/-
    - maximum ratepersqft is 172541/-
    - minimum ratepersqft is 100/-
    - 25 percentile is 1018 i.e, 25% of ratepersqft's lie below 1018/-
    - 50 percentile is 2222 i.e, 50% of ratepersqft's lie below 2222/-
    - 75 percentile is 4800 i.e, 75% of ratepersqft's lie below 4800/-
    - standard deviation(std) is 3850.07
    
 - **area_insqft**:
 
    - mean area is 2360.06 sqft
    - maximum area is 215278 sqft.
    - minimum area is 100 sqft.
    - 25 percentile is 1290 sqft i.e, 25% of area's lie below 1290 sqft.
    - 50 percentile is 1674 sqft i.e, 50% of area's lie below 1674 sqft.
    - 75 percentile is 2250 sqft i.e, 75% of area's lie below 2250 sqft.
    - standard deviation(std) is 4190.33
    
 - **agent_rating**:
 
    - mean of rating is 3.5
    - maximum rating is 5.0
    - minimum rating is 0.0
    - 25 percentile is 3.4 i.e, 25% of rating's lie below 3.4
    - 50 percentile is 4.6 i.e, 50% of rating's lie below 4.6
    - 75 percentile is 4.9 i.e, 75% of rating's lie below 4.9
    - standard deviation(std) is 1.95
    

In [185]:
# Drops the duplicate entires in the dataset.

d2=d2.drop_duplicates()

In [186]:
# As number of rows would vary we need to reset index.

d2=d2.reset_index()

In [187]:
# Dropping unnecessary columns in dataset.

d2=d2.drop(labels='index',axis=1)

In [188]:
d2.shape

(25715, 7)

### Observations:
- Before deleting duplicate entries total datapoints were **40260**
- After deleting duplicate entries total datapoints reduced to **25715**
- That is we have **14545** (40260-25715) duplicate entries.

In [63]:
d2.head()

,title,location,price(L),rate_persqft,area_insqft,building_status,agent_rating
0,Residential Plot,Maheshwaram,45.00,1111,4050,New,4.6
1,Residential Plot,Kondakal,34.00,2361,1440,New,3.7
2,Residential Plot,Bibinagar,14.19,944,1503,New,4.9
3,Residential Plot,Gachibowli,330.00,3333,9900,New,0.0
4,Residential Plot,Shadnagar,6.72,466,1440,New,4.7


In [64]:
# Returns last 5 rows from dataset (default value of tail is 5).

d2.tail()

,title,location,price(L),rate_persqft,area_insqft,building_status,agent_rating
25710,2 BHK Apartment,Chandanagar,47.00,4563,1030,Ready to move,0.0
25711,2 BHK Independent House,Kundanpally,25.74,3961,650,Ready to move,0.0
25712,3 BHK Apartment,Peeramcheru,81.00,4500,1800,Ready to move,0.0
25713,2 BHK Apartment,Nizampet,34.50,3833,900,Under Construction,0.0
25714,2 BHK Apartment,Chandanagar,40.00,4444,900,Under Construction,0.0


In [66]:
# To check whether there are any null values.

d2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25715 entries, 0 to 25714
Data columns (total 7 columns):
title              25715 non-null object
location           25715 non-null object
price(L)           25715 non-null float64
rate_persqft       25715 non-null int64
area_insqft        25715 non-null int64
building_status    25715 non-null object
agent_rating       25715 non-null float64
dtypes: float64(2), int64(2), object(3)
memory usage: 1.4+ MB


### Observation:
- we can see from info that there are no null value containing columns.

In [67]:
# The function 'values' returns the details of rows containing maximum and minimum values of a particular column.

def values(x):
    max_=d2[x].idxmax()
    max_details=pd.DataFrame(d2.loc[max_])
    
    min_=d2[x].idxmin()
    min_details=pd.DataFrame(d2.loc[min_])
    
    both=pd.concat([max_details,min_details],axis=1)
    
    return both

In [68]:
values('price(L)')

,10937,7860
title,4 BHK Independent House,Residential Plot
location,Banjara Hills,Vanasthalipuram
price(L),7000,0
rate_persqft,172541,100
area_insqft,4058,450
building_status,Ready to move,New
agent_rating,3.6,4.4


### Observations:
- Maximum price is 7000 L i.e, 70 cr for a 4 BHK Independent House in Banjara Hills with an area of 4058 sqft and rate per sqft   is 172541.
- Minimum price is 0 L which cant be true, we can find price from area and rate per sqft which are available for that property

In [192]:
# Returns all the properties whose price is 0 L.

d3=d2[(d2['price(L)']==0)]
d3

,title,location,price(L),rate_persqft,area_insqft,building_status,agent_rating
7860,Residential Plot,Vanasthalipuram,0.0,100,450,New,4.4
9992,Residential Plot,Jubilee Hills,0.0,333,150,New,2.9
13947,Residential Plot,Kukatpally,0.0,213,150,New,2.4
14320,Residential Plot,Nagole Bandlaguda Road,0.0,106,150,New,2.0


### Observations:
- There are four properties whose price value is missing and for these properties we hava ratepersqft and area of property from   which we can calculate price.

In [193]:
#Returns index positions of those properties whose price is 0 L.

n = d3[d3['price(L)']==0].index.tolist() 
n

[7860, 9992, 13947, 14320]

In [194]:
#This code multiplies ratepersqft and area and returns the price for the properties using index values, whose prices are 0 L.

import numpy as np
n = d3[d3['price(L)']==0].index.tolist() # Returns index values of rows whose price is 0 L.
price1=[] 
c=0
for i in (d3['price(L)']):
    if i == 0:
        a=d3.loc[n[c],'rate_persqft'] # Returns the value of ratepersqft at nth location.
        b=d3.loc[n[c],'area_insqft'] # Returns value of area at nth location.
        m=np.round(float((a*b)/100000),2) # multiplies area and ratepersqft and coverts type of output to float.
        d2.loc[n[c], 'price(L)'] = m # Appends the obtained value from mulitplication to nth location.
        c+=1
        price1.append(m)

In [195]:
price1 # Prices obatined after multiplying ratepersqft and area.

[0.45, 0.5, 0.32, 0.16]

In [196]:
# Returns rows whose price value is 0 L.

d2[d2['price(L)']==0]

,title,location,price(L),rate_persqft,area_insqft,building_status,agent_rating


### Observation:
- From above we can see that there are no more properties having price as 0 L.

In [79]:
# Returns the rows from particular locations

d2.iloc[[7860, 9992, 13947, 14320 ] , : ]

,title,location,price(L),rate_persqft,area_insqft,building_status,agent_rating
7860,Residential Plot,Vanasthalipuram,0.45,100,450,New,4.4
9992,Residential Plot,Jubilee Hills,0.50,333,150,New,2.9
13947,Residential Plot,Kukatpally,0.32,213,150,New,2.4
14320,Residential Plot,Nagole Bandlaguda Road,0.16,106,150,New,2.0


### Observation:
- We can observe that our logic for 0 L price values is correct as price is rightly appended with the respective rows.

In [159]:
d5=pd.read_excel("dup1.xlsx",index_col=0)

In [208]:
#checking whether there's any change in mean and other values after data cleaning.

print("\t\t\t Before Cleaning")
print(d5.describe())
print('-'*63)
print('*'*63)
print('-'*63)
print("\t\t\t After Cleaning")
print(d2.describe())

			 Before Cleaning
           price(L)   rate_persqft    area_insqft  agent_rating
count  40260.000000   40260.000000   40260.000000  40260.000000
mean      65.331670    3297.212072    2360.068728      3.574724
std      126.247236    3850.078324    4190.331350      1.953369
min        0.000000     100.000000     100.000000      0.000000
25%       20.000000    1018.000000    1290.000000      3.400000
50%       37.170000    2222.000000    1674.000000      4.600000
75%       78.630000    4800.000000    2250.000000      4.900000
max     7000.000000  172541.000000  215278.000000      5.000000
---------------------------------------------------------------
***************************************************************
---------------------------------------------------------------
			 After Cleaning
           price(L)   rate_persqft    area_insqft  agent_rating
count  25715.000000   25715.000000   25715.000000  25715.000000
mean      68.963410    3491.056621    2328.315497      3.363387
s

### Different b/w Observations:

 - **price(L)**:
 
    - mean of prices changed from 65.33 to 68.96 L.
    - minimum price from 0 L to 0.16 L.
    - 25 percentile stays same 20 L i.e, 25% of prices lie below 20 L.
    - 50 percentile changed from 37.17  to 38.98 L i.e, 50% of prices lie below 38.98 L.
    - 75 percentile changed from 78.63  to 76.60 L i.e, 50% of prices lie below 76.60 L.
    - standard deviation(std) raised to 140.98 from 126.24
 
 - **rate_persqft**:
 
    - mean ratepersqft from 3297.21 to 3491/-
    - 25 percentile from 1018 to 1111/-
    - 50 percentile from 2222 to 2556/-
    - 75 percentile from 4800 to 4962/-
    - standard deviation(std) from 3850.07 to 4218.05
    
 - **area_insqft**:
 
    - mean area from 2360.06 to 2328.31
    - 25 percentile from 1290 sqft to 1250 sqft.
    - 50 percentile from 1674 sqft to 1645 sqft.
    - 75 percentile stays same i.e, 2250 sqft.
    - standard deviation(std) from 4190.33 to 4836.96
    
 - **agent_rating**:
 
    - mean of rating from 3.5 to 3.3
    - 25 percentile from 3.4 to 0.0
    - 50 percentile from 4.6 to 4.5
    - 75 percentile from 4.9 to 4.8
    - standard deviation(std) from 1.95 to 2.01

In [85]:
# Importing the cleaned data into a new excel file.

writer = pd.ExcelWriter('current.xlsx',engine ='xlsxwriter')
d3.to_excel(writer, 'Sheet1')
writer.save()

In [86]:
df=pd.read_excel('current.xlsx',index_col=0)

In [88]:
df.shape

(25715, 7)